In [1]:
import torch
import numpy as np
import tqdm
from assist_chall import assistchall_process
import pandas as pd
from Freyja_LPKT import LPKTNet_copy
from Freyja_LPKT import LPKT_dataloader
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import sklearn.metrics as metrics
from assist_2012 import assist2012_process
from ednet import ednet_process

In [3]:
path = 'assist_chall/assist_chall_4LPKT.csv'
data_inf, data_sum = assistchall_process.data_split(pd.read_csv(path, encoding="utf-8", low_memory=True))

Student Number: 1709
Exercise Number: 3162
Skill Number: 102


100%|██████████| 1709/1709 [00:00<00:00, 89559.40it/s]


In [14]:
raw_data = pd.read_csv(path, encoding="utf-8", low_memory=True)
raw_data

,studentId,problemId,skill,startTime,endTime,timeTaken,correct
0,1,1118,0,1096470301,1096470350,49.0,0
1,1,1119,0,1096470350,1096470354,4.0,1
2,1,1120,1,1096470354,1096470360,6.0,0
3,1,1120,1,1096470360,1096470378,18.0,0
4,1,1121,1,1096470378,1096470380,2.0,1
...,...,...,...,...,...,...,...
942776,1709,1712,61,1147447464,1147447468,4.0,1
942777,1709,1713,61,1147447468,1147447539,71.0,0
942778,1709,1713,61,1147447539,1147447543,4.0,0
942779,1709,1713,61,1147447543,1147447546,3.0,1


In [15]:
raw_data = raw_data.sort_values(by=['startTime'])
raw_data = np.array(raw_data)
raw_stu_id = raw_data[:, 0]
raw_exercise_id = raw_data[:, 1]
raw_skill = raw_data[:, 2]

stu_id = np.unique(raw_stu_id)
exercise_id = np.unique(raw_exercise_id)
skill_id = np.unique(raw_skill)


In [38]:
stu_num = len(stu_id)
exercise_num = len(exercise_id)
skill_num = len(skill_id)
print("Student Number:", stu_num)
print("Exercise Number:", exercise_num)
print("Skill Number:", skill_num)
min(exercise_id)


Student Number: 1709
Exercise Number: 3162
Skill Number: 102


1.0

In [32]:
q_matrix = np.zeros([exercise_num+1, skill_num])
for i in range(len(raw_data)):
    q_matrix[int(raw_exercise_id[i]), int(raw_skill[i])] = 1
q_matrix = torch.from_numpy(q_matrix)


In [37]:
q_matrix.shape

torch.Size([3163, 102])